# Cross-validation and hyperparameter tuning

Nos notebooks anteriores, vimos duas abordagens para ajustar hiperparâmetros:
por meio de pesquisa em grade e pesquisa aleatória.

Neste bloco de notas, mostraremos como combinar tais pesquisas de hiperparâmetros
com uma validação cruzada.

## Nosso modelo preditivo

Vamos recarregar o conjunto de dados como fizemos anteriormente:

In [1]:
from sklearn import set_config

set_config(display="diagram")

In [2]:
import pandas as pd

adult_census = pd.read_csv("adult-census.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns=['ID','fnlwgt:','education-num:'])

Extraímos a coluna que contém o target.

In [3]:
target_name = "class"
target = adult_census[target_name]
target

0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
32556    <=50K
32557     >50K
32558    <=50K
32559    <=50K
32560     >50K
Name: class, Length: 32561, dtype: object

Tiramos de nossos dados o alvo e a coluna `" education-num "` que
duplica a informação com colunas `" educação "`.

In [4]:
data = adult_census.drop(columns=[target_name])
data['native-country:'] = data['native-country:'].replace(['Holand-Netherlands'], 'United-States')
data.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


Depois que o conjunto de dados é carregado, nós o dividimos em conjuntos de treinamento e teste.

In [5]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

Vamos criar o mesmo pipeline preditivo visto no grid-search
seção.

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)

categorical_preprocessor = OrdinalEncoder()

preprocessor = ColumnTransformer([
    ('cat-preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough', sparse_threshold=0)
preprocessor

ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                  transformers=[('cat-preprocessor', OrdinalEncoder(),
                                 ['workclass', 'education:', 'marital-status:',
                                  'occupation:', 'relationship:', 'race:',
                                  'sex:', 'native-country:'])])

In [7]:
# for the moment this line is required to import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier",
     HistGradientBoostingClassifier(random_state=42, max_leaf_nodes=4))])
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('cat-preprocessor',
                                                  OrdinalEncoder(),
                                                  ['workclass', 'education:',
                                                   'marital-status:',
                                                   'occupation:',
                                                   'relationship:', 'race:',
                                                   'sex:',
                                                   'native-country:'])])),
                ('classifier',
                 HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                random_state=42))])

## Incluir uma pesquisa de hiperparâmetro em uma validação cruzada

Como mencionado anteriormente, usando uma única divisão de teste de trem durante a pesquisa de grade
não dá nenhuma informação sobre as diferentes fontes de variações:
variações em termos de pontuação de teste ou valores de hiperparâmetros.

Para obter informações confiáveis, a pesquisa de hiperparâmetros precisa ser aninhada
dentro de uma validação cruzada.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">Para limitar o custo computacional, afetamos<tt class="docutils literal">cv</tt> para um número inteiro baixo. Em
prática, o número de vezes deve ser muito maior.</p>
</div>

In [8]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__learning_rate': (0.05, 0.1),
    'classifier__max_leaf_nodes': (30, 40)}
model_grid_search = GridSearchCV(model, param_grid=param_grid,
                                 n_jobs=4, cv=2)

cv_results = cross_validate(
    model_grid_search, data, target, cv=3, return_estimator=True)

Executar a validação cruzada acima nos dará uma estimativa do
pontuação de teste.

In [9]:
scores = cv_results["test_score"]
print(f"Accuracy score by cross-validation combined with hyperparameters "
      f"search:\n{scores.mean():.3f} +/- {scores.std():.3f}")

Accuracy score by cross-validation combined with hyperparameters search:
0.870 +/- 0.002


Os hiperparâmetros em cada dobra são potencialmente diferentes, pois aninhamos
a pesquisa em grade na validação cruzada. Assim, verificando a variação do
os hiperparâmetros nas dobras também devem ser analisados.

In [10]:
for fold_idx, estimator in enumerate(cv_results["estimator"]):
    print(f"Best parameter found on fold #{fold_idx + 1}")
    print(f"{estimator.best_params_}")

Best parameter found on fold #1
{'classifier__learning_rate': 0.1, 'classifier__max_leaf_nodes': 30}
Best parameter found on fold #2
{'classifier__learning_rate': 0.1, 'classifier__max_leaf_nodes': 30}
Best parameter found on fold #3
{'classifier__learning_rate': 0.05, 'classifier__max_leaf_nodes': 30}


Obter modelos com hiperparâmetros instáveis seria um problema na prática.
Na verdade, seria difícil defini-los.

Neste bloco de notas, vimos como combinar a pesquisa de hiperparâmetros com
validação cruzada.